# Introduction

This notebook tries to cover most of the concepts learnt during the course, and applies them using PyTorch

1. TODO: Input processing
   1. Data Augmentation
2. Network Topology
   1. Layers
   2. Activation Function
   3. Loss Function
3. Network Training (Optimization)
   1. Hyperparameter Tuning
   2. Weight Initialization

# Input Processing

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets 
import torchvision.transforms as transforms

In [5]:
# Hyperparameters

input_size = 784        # Number of input neurons (image pixels): 28x28 = 784
hidden_size = 400       # Number of hidden neurons
out_size = 10           # Number of classes (0-9) 
epochs = 10             # How many times we pass our entire dataset into our network 
batch_size = 100        # Input size of the data during one iteration 
learning_rate = 0.001   # How fast we are learning

In [10]:
train_dataset = datasets.MNIST(root='./data',
                           train=True,
                           transform=transforms.ToTensor(),
                           download=True)

test_dataset = datasets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())

train_dataset.data.shape
test_dataset.data.shape

torch.Size([10000, 28, 28])

We check the shape of downloaded training dataset, and find it has 60K images of 28x28, while test dataset has 10K images.

Now, we use [PyTorch DataLoader](https://pytorch.org/docs/stable/data.html) to load the training and test datasets

> **Batch Size** indicates the number of input samples used for a single **Iteration**

In [7]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

# Network Topology

Now, we define the network, as a class derived from [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module)

> [ReLU activation function](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html#torch.nn.ReLU) is used, and [Kaiming Normal Weights Initialization](https://pytorch.org/docs/stable/nn.init.html#torch.nn.init.kaiming_normal_) is used

In [12]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, out_size):
        super(Net, self).__init__()                    
        self.fc1 = nn.Linear(input_size, hidden_size)    #First Layer                           
        self.fc2 = nn.Linear(hidden_size, hidden_size)      #Second Layer Activation
        self.fc3 = nn.Linear(hidden_size, out_size)
        self.relu = nn.ReLU()
        self.init_weights()
        
    def init_weights(self):
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.kaiming_normal_(self.fc2.weight)
        # OPEN: Why output layer weights are not initialized?

    def forward(self, x):                          
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

Now we define loss function and optimizer

In [13]:
# Create an instance of the NN class defined above
net = Net(input_size, hidden_size, out_size)

# Check if CUDA can be used to speed up
# TODO: Can we use OpenVINO?
CUDA = torch.cuda.is_available()
if CUDA:
    net = net.cuda()

#The loss function. The Cross Entropy loss comes along with Softmax. Therefore, no need to specify Softmax as well
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

# Network Training

Mathematically, we're trying to find the optimal set of network weights, which lead to minimum loss (actual output - expected output)

> For each batch of inputs, the outputs are evaluated, and loss in prediction calculated. The averaged loss is used to back-propagate the error and adjust the weights

> Each epoch uses the entire training data. As more epochs complete, the expectation is that training error reduces

In [14]:
#Train the network
for epoch in range(epochs):
    correct_train = 0
    running_loss = 0
    for i, (images, labels) in enumerate(train_loader):   
        #Flatten the image from size (batch,1,28,28) --> (100,1,28,28) where 1 represents the number of channels (grayscale-->1),
        # to size (100,784) and wrap it in a variable
        images = images.view(-1, 28*28)    
        if CUDA:
            images = images.cuda()
            labels = labels.cuda()
            
        # Evaluate the network for batch of inputs
        outputs = net(images)       

        _, predicted = torch.max(outputs.data, 1)                                              
        correct_train += (predicted == labels).sum()

        # Evaluate the loss using specified loss function
        loss = criterion(outputs, labels)                 # Difference between the actual and predicted (loss function)
        running_loss += loss.item()

        # Back propagation to adjust the weights for calculated loss
        optimizer.zero_grad() 
        loss.backward()                                   # Backpropagation
        optimizer.step()                                  # Update the weights
        
    print('Epoch [{}/{}], Training Loss: {:.3f}, Training Accuracy: {:.3f}%'.format
          (epoch+1, epochs, running_loss/len(train_loader), (100*correct_train.double()/len(train_dataset))))
print("DONE TRAINING!")

Epoch [1/10], Training Loss: 0.232, Training Accuracy: 93.320%
Epoch [2/10], Training Loss: 0.087, Training Accuracy: 97.338%
Epoch [3/10], Training Loss: 0.055, Training Accuracy: 98.227%
Epoch [4/10], Training Loss: 0.039, Training Accuracy: 98.755%
Epoch [5/10], Training Loss: 0.028, Training Accuracy: 99.092%
Epoch [6/10], Training Loss: 0.023, Training Accuracy: 99.283%
Epoch [7/10], Training Loss: 0.020, Training Accuracy: 99.362%
Epoch [8/10], Training Loss: 0.019, Training Accuracy: 99.372%
Epoch [9/10], Training Loss: 0.016, Training Accuracy: 99.468%
Epoch [10/10], Training Loss: 0.013, Training Accuracy: 99.580%
DONE TRAINING!
